In [32]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import os
import random

random_items_list = []

class Market:
    def __init__(self):
        self.items = {
            'Nồi': (7, 0.7), 'Cốc': (3, 0.25), 'Chén': (2, 0.2), 'Chảo': (6, 0.45), 
            'Rổ': (2, 0.15), 'Thớt': (8, 1), 'Sách': (5, 0.3), 'Bút': (1, 0.1), 
            'Giày': (20, 0.35), 'Kẹp tóc': (2, 0.05), 'Áo lông': (30, 0.8), 
            'Nón bảo hiểm': (15, 0.3), 'Balo': (18, 0.4), 'Lò vi sóng': (100, 4), 
            'Máy sấy tóc': (32, 0.45), 'Tranh treo tường': (23, 0.5), 
            'Đồng hồ treo tường': (15, 0.38), 'Chậu cây': (16, 1.5), 
            'Ốp điện thoại': (2.5, 0.15), 'Gọng kính': (20, 0.1)
        }
        self.images = self.load_images()
        self.max_weight = sum(weight for _, weight in self.items.values())

    def load_images(self):
        images = {}
        for item in self.items:
            try:
                png_path = rf'C:\Users\Gia Bảo\Downloads\Items\{item}.png'
                jpg_path = rf'C:\Users\Gia Bảo\Downloads\Items\{item}.jpg'

                if os.path.exists(png_path):
                    images[item] = ImageTk.PhotoImage(Image.open(png_path).resize((50, 50)))
                elif os.path.exists(jpg_path):
                    images[item] = ImageTk.PhotoImage(Image.open(jpg_path).resize((50, 50)))
                else:
                    print(f"No image found for item: {item}")
            except Exception as e:
                print(f"Error loading image for item {item}: {e}")
        return images

    def get_items(self):
        return self.items
    
    def get_images(self):
        return self.images
    
    def get_max_weight(self):
        return self.max_weight

def generate_random_items(items, count):
    random_items = []
    item_counts = {item: 0 for item in items}

    
    while len(random_items) < 10:
        item = random.choice(list(items.keys()))
        if item not in random_items:
            random_items.append(item)
            item_counts[item] += 1

   
    while len(random_items) < count:
        item = random.choice(list(items.keys()))
        if item_counts[item] < 3:
            random_items.append(item)
            item_counts[item] += 1

    return random_items

def generate_solution(items, capacity, items_to_use):
    solution = []
    current_weight = 0
    
    available_items = items_to_use.copy()
    
    while current_weight < capacity and available_items:
        item = random.choice(available_items)
        item_weight = items[item][1]
        if current_weight + item_weight <= capacity:
            solution.append(item)
            current_weight += item_weight
        available_items.remove(item) 
    
    return solution

def evaluate_solution(items, solution):
    total_price = sum(items[item][0] for item in solution)
    total_weight = sum(items[item][1] for item in solution)
    return total_price, total_weight

def generate_neighbors(items, solution, capacity, items_to_use):
    neighbors = []
    available_items = list(set(items_to_use) - set(solution))
    for i in range(len(solution)):
        for item in available_items:
            neighbor = solution.copy()
            neighbor[i] = item
            total_weight = sum(items[it][1] for it in neighbor)
            if total_weight <= capacity:
                neighbors.append(neighbor)
    return neighbors

def steepest_ascent_hill_climbing(items, capacity, max_iterations, items_to_use):
    best_solution = []
    best_price = float('-inf')
    
    for _ in range(max_iterations):
        current_solution = generate_solution(items, capacity, items_to_use)
        current_price, current_weight = evaluate_solution(items, current_solution)
        
        while True:
            neighbors = generate_neighbors(items, current_solution, capacity, items_to_use)
            if not neighbors:
                break
            
            best_neighbor = current_solution
            best_neighbor_price, best_neighbor_weight = current_price, current_weight
            
            for neighbor in neighbors:
                neighbor_price, neighbor_weight = evaluate_solution(items, neighbor)
                if (neighbor_price > best_neighbor_price) or (neighbor_price == best_neighbor_price and neighbor_weight > best_neighbor_weight):
                    best_neighbor = neighbor
                    best_neighbor_price, best_neighbor_weight = neighbor_price, neighbor_weight
            
            if best_neighbor == current_solution:
                break
            
            current_solution = best_neighbor
            current_price, current_weight = best_neighbor_price, best_neighbor_weight
        
        if current_price > best_price:
            best_price = current_price
            best_solution = current_solution
    
    return best_solution, best_price

def generate_random_items_display():
    global random_items_list
    random_items_list = generate_random_items(market.get_items(), 15)
    random_items_text = " | ".join(f"{item} x{random_items_list.count(item)}" for item in set(random_items_list))
    random_items_label.config(text=random_items_text)
    
   
    max_capacity = sum(market.get_items()[item][1] for item in random_items_list)
    max_capacity = round(max_capacity, 1)  # Round to one decimal place
    max_capacity_label.config(text=f"Max Capacity: {max_capacity} kg")

   
    canvas.delete("all")

def start_algorithm():
    try:
        capacity = float(entry_capacity.get())
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a valid number for capacity.")
        return

   
    items = {name: market.get_items()[name] for name in random_items_list} if random_items_list else market.get_items()
    items_to_use = random_items_list if random_items_list else list(items.keys())
    
    max_iterations = 500
    final_solution, final_price = steepest_ascent_hill_climbing(items, capacity, max_iterations, items_to_use)
    
    canvas.delete("all") 
    
    x_offset = 10
    y_offset = 10
    for item in final_solution:
        price, weight = items[item]
        if item in market.get_images():
            canvas.create_image(x_offset, y_offset, anchor=tk.NW, image=market.get_images()[item])
            label_text = f"{item}\nPrice: {price}\nWeight: {weight}"
            canvas.create_text(x_offset + 60, y_offset, anchor=tk.NW, text=label_text)
            x_offset += 150  # Adjust x-coordinate for next item
            if x_offset >= canvas.winfo_width() - 150:
                x_offset = 10
                y_offset += 100  # Adjust y-coordinate for next row

    total_price_label.config(text=f"Total price: {final_price}")

root = tk.Tk()
root.title("Knapsack Problem Solver")


root.tk_setPalette(background='#add8e6')

frame_input = tk.Frame(root)
frame_input.pack(pady=10)

market = Market()
max_weight = market.get_max_weight()

tk.Label(frame_input, text="Enter the capacity of the bag in kg:").pack(side=tk.LEFT)
entry_capacity = tk.Entry(frame_input)
entry_capacity.pack(side=tk.LEFT)
entry_capacity.focus_set()

# Label to display max capacity
max_capacity_label = tk.Label(frame_input, text=f"Max Capacity: {max_weight} kg")
max_capacity_label.pack(side=tk.LEFT)

btn_start = tk.Button(frame_input, text="Start", command=start_algorithm)
btn_start.pack(side=tk.LEFT, padx=10)

# Button to generate random items
btn_generate_random = tk.Button(frame_input, text="Generate Random Items", command=generate_random_items_display)
btn_generate_random.pack(side=tk.LEFT, padx=10)

# Label to display random items
random_items_label = tk.Label(root, text="Random Items: None")
random_items_label.pack(pady=10)

# Create canvas to display the solution
canvas = tk.Canvas(root, width=800, height=400, bg="white")
canvas.pack(pady=10)

# Label to display the total price
total_price_label = tk.Label(root, text="Total price: 0")
total_price_label.pack(pady=10)

root.mainloop()
